In [15]:
import numpy as np
from math import cos, sin, radians, sqrt, pi, atan, tan


In [75]:
class Elemento():
    def __init__(self,elemento, base, altura, moduloElastico, coordenada_xi, coordenada_yi, coordenada_xf, coordenada_yf, conexion):
        
        self.Elem = elemento
        self.b = base
        self.h = altura
        self.E = moduloElastico
        self.cxi = coordenada_xi
        self.cyi = coordenada_yi
        self.cxf = coordenada_xf
        self.cyf = coordenada_yf
        self.conex = conexion
        # Propiedades del elemento
        self.A = self.Area()
        self.I = self.Inercia()
        self.L = self.Longitud()
        self.ang = self.Angulo()
        self.l = self.coseno()
        self.m = self.seno()

    # Propiedades del elemento

    def Area(self):
        return self.b * self.h
    def Inercia(self):
        return (self.b * self.h**3) / 12
    def Longitud(self):
        return sqrt((self.cxf - self.cxi)**2 + (self.cyf - self.cyi)**2)
    def Angulo(self):
        b = (self.cxf - self.cxi)
        h = (self.cyf - self.cyi)
        try:
            angulo = atan(h/b)*180/pi
        except ZeroDivisionError:
            angulo = 90
        return angulo

    def seno(self):
        return sin(radians(self.ang))

    def coseno(self):
        return cos(radians(self.ang))        



In [76]:
E  = 219498.39

tbl_Elem = [
    
  # Elem, Ni, Nf, b, h, E,
    ["1", 1, 2, 40, 40, E],
    ["2", 2, 3, 40, 40, E],
    ["3", 4, 3, 40, 40, E],

    ]
    
tbl_Nod = [
    #Nodo, cx, cy
    [1, 0, 0],
    [2, 0, 500],
    [3, 1000, 767.95],
    [4, 1000, 0],

    ]
        #El, Ni, Nf
#print(nodos[:])
tbl_Fuer = [
    #Nod, fx, fy, Mz
    [2,500, 0, 0],
  
    ]

tbl_Fuer_W = [
    #Elem, Li,Lf, Wi, Wf
    ["2", 0, 1035, -1, -1],

    ]

tbl_Rest = [
        #Tipo de Apoyo, Nodo
        # Simple
        # Articulado
        # Empotrado
    ["Empotrado", 1],
    ["Empotrado", 4,],

    ]
"""====Diccionarios===="""
# ==============Diccionarios de elementos==================#

diccionarioE = {}

for i in range(len(tbl_Elem)):
    Elem = tbl_Elem[i][0]
    Ni = tbl_Elem[i][1]
    Nf = tbl_Elem[i][2]
    b = tbl_Elem[i][3]
    h = tbl_Elem[i][4]
    E = tbl_Elem[i][5]   
    diccionarioE[Elem] = [Ni, Nf, b, h, E]

# ==============Diccionarios de nodos==================#
    
diccionarioN = {}

for i in range(len(tbl_Nod)):
    Nod = tbl_Nod[i][0]
    cx = tbl_Nod[i][1]
    cy = tbl_Nod[i][2]
    diccionarioN[Nod] = [cx, cy]
    

In [77]:
diccionarioN

{1: [0, 0], 2: [0, 500], 3: [1000, 767.95], 4: [1000, 0]}

In [78]:
elementos = [] # Almacena todos los elementos con sus propiedades

def ext_elem(): # Elementos fuera del analisis

    for i in range(len(tbl_Elem)):
        Ni = tbl_Elem[i][1]
        Nf = tbl_Elem[i][2]
        b = tbl_Elem[i][3]
        h = tbl_Elem[i][4]
        Ele = tbl_Elem[i][0]    
        conexion = [Ele, Ni, Nf]
        xi = diccionarioN[Ni][0]
        yi = diccionarioN[Ni][1]
        xf = diccionarioN[Nf][0]
        yf = diccionarioN[Nf][1]     
        barra = Elemento(Ele,b, h, E, xi, yi, xf, yf, conexion)
        elementos.append(barra) # Almacena elemento
    return elementos


elementos = ext_elem()
        

In [79]:
diccionarioN


{1: [0, 0], 2: [0, 500], 3: [1000, 767.95], 4: [1000, 0]}

# SUB-DIVISIONES

In [80]:
sub = 2
coordenadas = []
coor_sub_elementos = []

for i in range(len(tbl_Elem)):
    coord_p = [] # Guardado parcial
    Elemento = elementos[i]
    L = Elemento.L
    L_unitaria = L/sub
    
    cxi = Elemento.cxi
    cyi = Elemento.cyi
    cxf = Elemento.cxf
    cyf = Elemento.cyf
    
    ang = Elemento.ang
    for j in range(sub+1):
        
        # Elementos horizontales
        if angulo == 0: 
            cy = cyi
            
            if cyf-cyi <0:
                #print(i+1,"Yes",cyi,cyf )
                
                cx = round(cxi - L_unitaria*j,5)
            else:
                cx = round(cxi + L_unitaria*j,5)
                
            #cx = cxi + L_unitaria*j
            coord_p.append([cx,cy])
            
        # Elementos verticales
        elif ang == 90:
            
            cx = Elemento.cxi
            
            if cyf-cyi <0:
                #print(i+1,"Yes",cyi,cyf )
                cy = Elemento.cyi - round(L_unitaria*j,5)
            else:
                cy = Elemento.cyi + round(L_unitaria*j,5)
            coord_p.append([cx,cy])
                
        # Elementos inclinados crecientes
        elif ang > 0 and ang < 90:
            """estos fracmento forman l_uni en un triangulo rectangulo"""
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(ang))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly_uni/tan(radians(ang)) # Longitud del fracmento en x
            #cx = Elemento.cxi + round(Lx_uni*j,5)
            #cy = Elemento.cyi + round(Ly_uni*j,5)
            
            #si la diferencia  entre las coordenadas de los nodos es negativa 
            # entonces corrige o invierte la o peracio + o -
            # Por ejem: Nf = (0,10), Ni = (20,10) x = (0-20), y = (10-10) como en x es negativa se debe restar L_uni..
            if cyf-cyi <0:
                print(i+1,"Yes",cyi,cyf )
                cy =round( Elemento.cyi - Ly_uni*j,5)
            else:
                cy = round(Elemento.cyi + Ly_uni*j,5)
                
            if cxf-cxi <0:
                print(i+1,"Yes",cyi,cyf )
                cx = round(Elemento.cxi - Lx_uni*j,5)
            else:
                cx = round(Elemento.cxi + Lx_uni*j,5)
                
            coord_p.append([cx,cy])
            
            #print("j=",j, "Ly= ",Ly_uni,"Lx= ",Lx_uni, "cx= ",cx, "cy= ", cy)
                
        # Elementos inclinados decrecientes
        elif ang < 0:
            """estos fracmento forman l_uni en un triangulo rectangulo"""
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(ang))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly_uni/tan(radians(ang)) # Longitud del fracmento en x
            
            cx = Elemento.cxi - round(Lx_uni*j,5)
            cy = Elemento.cyi - round(Ly_uni*j,5)
               
            if cxf-cxi <0:
                print(i+1,"Yes",cyi,cyf )
                cx = round(Elemento.cxi + Lx_uni*j,5)
            else:
                cx = round(Elemento.cxi - Lx_uni*j,5)
            
            if cyf-cyi <0:
                #print(i+1,"Yes",cyi,cyf )
                cy =round( Elemento.cyi + Ly_uni*j,5)
            else:
                cy = round(Elemento.cyi - Ly_uni*j,5)
            coord_p.append([cx,cy])
            
        print(cx,cy)
        if [cx, cy] not in coordenadas: # Se chequea si la coord ya esta incluida
            coordenadas.append([cx,cy])
    coor_sub_elementos.append(coord_p)
coor_sub_elementos
coordenadas

0 0.0
0 250.0
0 500.0
0.0 500.0
500.0 633.975
1000.0 767.95
1000 0.0
1000 383.975
1000 767.95


[[0, 0.0],
 [0, 250.0],
 [0, 500.0],
 [500.0, 633.975],
 [1000.0, 767.95],
 [1000, 0.0],
 [1000, 383.975]]

In [81]:
tbl_sub_nodos = []
dic_sub_nodos = {}
id_nodo = 0
for n in range(len(coordenadas)):
    cx, cy = coordenadas[n]
    id_nodo += 1
    tbl_sub_nodos.append([id_nodo, cx, cy])
    dic_sub_nodos[id_nodo] = [cx, cy]
#tbl_sub_nodos 

dic_sub_nodos 

{1: [0, 0.0],
 2: [0, 250.0],
 3: [0, 500.0],
 4: [500.0, 633.975],
 5: [1000.0, 767.95],
 6: [1000, 0.0],
 7: [1000, 383.975]}

In [82]:
tbl_sub_elementos = []
dic_sub_elem_x_Elem = {}
id_elementos = 0
for i in range(len(tbl_Elem)):
    Elemento = elementos[i]
    Elem = Elemento.Elem
    b = Elemento.b
    h = Elemento.h
    E = Elemento.E
    angulo = Elemento.ang
    sub_elementos_p = []
    for j in range(sub):
        id_elementos += 1
        
        ci = coor_sub_elementos[i][j]
        cf = coor_sub_elementos[i][j+1]

        for m in range(len(tbl_sub_nodos)):

            N = m+1   
            
            if dic_sub_nodos[N] == ci:
                Ni = N
            if dic_sub_nodos[N] == cf:
                Nf = N  
        tbl_sub_elementos.append([str(id_elementos), Ni, Nf, b, h, E])
        sub_elementos_p.append([str(id_elementos), Ni, Nf, b, h, E])
    dic_sub_elem_x_Elem[Elem] = sub_elementos_p


In [83]:
dic_sub_elementos = {}
for i in range(len(tbl_sub_elementos)):
    Elem = tbl_sub_elementos[i][0]
    Ni = tbl_sub_elementos[i][1]
    Nf = tbl_sub_elementos[i][2]
    b = tbl_sub_elementos[i][3]
    h = tbl_sub_elementos[i][4]
    E = tbl_sub_elementos[i][5]   
    dic_sub_elementos[Elem] = [Ni, Nf, b, h, E]

In [84]:
dic_sub_elementos

{'1': [1, 2, 40, 40, 219498.39],
 '2': [2, 3, 40, 40, 219498.39],
 '3': [3, 4, 40, 40, 219498.39],
 '4': [4, 5, 40, 40, 219498.39],
 '5': [6, 7, 40, 40, 219498.39],
 '6': [7, 5, 40, 40, 219498.39]}

In [85]:
class Sub_Elementos():
    def __init__(self,elemento, base, altura, moduloElastico, coordenada_xi, coordenada_yi, coordenada_xf, coordenada_yf, conexion):
        
        self.Elem = elemento
        self.b = base
        self.h = altura
        self.E = moduloElastico
        self.cxi = coordenada_xi
        self.cyi = coordenada_yi
        self.cxf = coordenada_xf
        self.cyf = coordenada_yf
        self.conex = conexion
        # Propiedades del elemento
        self.A = self.Area()
        self.I = self.Inercia()
        self.L = self.Longitud()
        self.ang = self.Angulo()
        self.l = self.coseno()
        self.m = self.seno()

    # Propiedades del elemento

    def Area(self):
        return self.b * self.h
    def Inercia(self):
        return (self.b * self.h**3) / 12
    def Longitud(self):
        return sqrt((self.cxf - self.cxi)**2 + (self.cyf - self.cyi)**2)
    def Angulo(self):
        b = (self.cxf - self.cxi)
        h = (self.cyf - self.cyi)
        try:
            angulo = atan(h/b)*180/pi
        except ZeroDivisionError:
            angulo = 90
        return angulo

    def seno(self):
        return sin(radians(self.ang))

    def coseno(self):
        return cos(radians(self.ang))        

In [86]:
sub_elementos = [] # Almacena todos los elementos con sus propiedades
for i in range(len(tbl_sub_elementos)):
    Ni = tbl_sub_elementos[i][1]
    Nf = tbl_sub_elementos[i][2]
    b = tbl_sub_elementos[i][3]
    h = tbl_sub_elementos[i][4]
    Ele = tbl_sub_elementos[i][0]    
    conexion = [Ele, Ni, Nf]
    xi = dic_sub_nodos[Ni][0]
    yi = dic_sub_nodos[Ni][1]
    xf = dic_sub_nodos[Nf][0]
    yf = dic_sub_nodos[Nf][1]     
    barra = Sub_Elementos(Ele,b, h, E, xi, yi, xf, yf, conexion)
    sub_elementos.append(barra) # Almacena elemento

In [87]:
# Obtener cargar externas para los sub elementos
tbl_sub_F_W = []

for i in range(len(tbl_Fuer_W)):
    Elemento = tbl_Fuer_W[i][0]
    wi = tbl_Fuer_W[i][3]
    wf = tbl_Fuer_W[i][4]
    L = 0
    
    for n in range(len(elementos)): # Extra la longitud del elemento sin sub...
        if Elemento == elementos[n].Elem:
            L = elementos[n].L

    for j in range(sub):
        
        Elem = dic_sub_elem_x_Elem[Elemento][j][0]
   
        ################################################
        ##################################################
        # Carga distibuida rectangular
        #print(wi, wf)
        if wi == wf:
            
            L_unitaria = L/sub*(j+1)
            #print(L_unitaria)
            sub_W = wi
            tbl_sub_F_W.append([Elem, L/sub*j, L_unitaria,sub_W , sub_W])
            
            
        #########################################################
        ##############################################################
            
        # Carga distibuida triangular decreciente
        
        if  abs(wi) > 0  and abs(wf) == 0:
            
            W = wi
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L 
            sub_wf = (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        
        # Carga distibuida triangular creciente
        if  abs(wf) > 0  and abs(wi) == 0:
            
            W = wf
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L 
            sub_wf = (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        # Carga distibuida trapezoidal decreciente
        if  abs(wi) > abs(wf) and abs(wf) > 0:
            
            W = (wi - wf)
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L + wf
            sub_wf = (W * L_unitaria * (j+1))/L + wf
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        # Carga distibuida trapezoidal creciente
        if  abs(wi) < abs(wf) and abs(wi) > 0:
            
            W = (wf - wi)
            L_unitaria = L/sub
            sub_wi = wi + (W * L_unitaria * j) /L
            sub_wf = wi + (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])


#tbl_sub_F_W      

# Consulta por el nodo real

In [88]:
def nodo_ireal(nodo):
    coordenadas_reales = diccionarioN[nodo]
    for i in range(len(tbl_sub_nodos)):
        sub_N = i+1
        if coordenadas_reales == dic_sub_nodos[sub_N]:
            Nod = sub_N
            return Nod

In [89]:
tbl_sub_Fuer = []

for j in range(len(tbl_Fuer)):
    Nod = tbl_Fuer[j][0]
    fx = tbl_Fuer[j][1]
    fy = tbl_Fuer[j][2]
    mz = tbl_Fuer[j][3]
    sub_N = nodo_ireal(Nod)
    tbl_sub_Fuer.append([sub_N, fx, fy, mz])
#tbl_sub_Fuer

In [90]:
tbl_sub_restricciones = []

for j in range(len(tbl_Rest)):
    N_rest = tbl_Rest[j][1]
    tipo = tbl_Rest[j][0]
    sub_N = nodo_ireal(N_rest)
    tbl_sub_restricciones.append([tipo, sub_N])
tbl_sub_restricciones

[['Empotrado', 1], ['Empotrado', 6]]

In [91]:
tbl_sub_elementos

[['1', 1, 2, 40, 40, 219498.39],
 ['2', 2, 3, 40, 40, 219498.39],
 ['3', 3, 4, 40, 40, 219498.39],
 ['4', 4, 5, 40, 40, 219498.39],
 ['5', 6, 7, 40, 40, 219498.39],
 ['6', 7, 5, 40, 40, 219498.39]]

In [92]:
['1', 1, 2, 40, 40, 219498.39],
 ['2', 2, 3, 40, 40, 219498.39],
 ['3', 3, 4, 40, 40, 219498.39],
 ['4', 4, 5, 40, 40, 219498.39],
 ['5', 5, 6, 40, 40, 219498.39],
 ['6', 6, 7, 40, 40, 219498.39]]

IndentationError: unexpected indent (1132126425.py, line 2)

In [93]:
tbl_sub_nodos

[[1, 0, 0.0],
 [2, 0, 250.0],
 [3, 0, 500.0],
 [4, 500.0, 633.975],
 [5, 1000.0, 767.95],
 [6, 1000, 0.0],
 [7, 1000, 383.975]]

In [72]:
tbl_sub_Fuer

[[3, 500, 0, 0]]

In [73]:
tbl_sub_F_W

[['3', 0.0, 517.6381947122914, -1, -1],
 ['4', 517.6381947122914, 1035.2763894245827, -1, -1]]

In [74]:
tbl_sub_restricciones

[['Empotrado', 1], ['Empotrado', 7]]

In [48]:
nodo_ireal(4)
#dic_sub_elem_x_Elem["8"]

7

{1: [0, 500.0],
 2: [0, 750.0],
 3: [0, 1000.0],
 4: [500.0, 633.975],
 5: [1000.0, 767.95],
 6: [1000, 0.0],
 7: [1000, 383.975]}

In [ ]:
#Pruebas

False

False

False

In [106]:
tbl_sub_restricciones == [['Empotrado', 1], ['Empotrado', 6], ['Empotrado', 11]]

True

In [26]:
nodo_ireal(4)